## Generate Data

In [ ]:
from scipy import signal
import numpy as np

In [ ]:
fs = 48e3
f0 = 20
f1 = 20e3
t = np.linspace(0, 60, 60*fs)

train_input = signal.chirp(t=t, f0=f0, t1=60, f1=f1, method='logarithmic') + np.random.normal(scale=5e-2, size=len(t))

fc = 2e3
sos = signal.butter(N=2, Wn=fc/fs, output='sos')
train_target = signal.sosfilt(sos, train_input)

# Declare Model
Example will only be shown for this model, the other models are analogous.

In [ ]:
import torch
model_type = "SVF" #or "TDF-II" or "LSS" or "DOPOZ"

device = torch.device('cpu')

if model_type == "SVF":
    from dsvf_models import DSVF
    model = DSVF().to(device)
elif model_type == 'TDF-II':
    from differentiable_tdf2_model import DTDFII
    model = DTDFII().to(device)
elif model_type == 'LSS':
    from linear_state_space_model import LinearStateSpaceModel
    model = LinearStateSpaceModel(num_states=2).to(device)
elif model_type == "DOPOZ":
    from differentiable_one_pole_one_zero import DOnePole
    model = DOnePole().to(device)
else:
    print("Please pick valid model type")

## Initialize Dataloader

In [ ]:
from iir_dataset import DIIRDataSet
from torch.utils.data import DataLoader

batch_size = 1024
sequence_length = 512

loader = DataLoader(dataset=DIIRDataSet(train_input, train_target, sequence_length), batch_size=batch_size, shuffle = True)

## Define optimizer and criterion

In [ ]:
import torch.nn as nn
from torch.optim import Adam

n_epochs = 1000
lr = 1e-3

optimizer = Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

criterion = nn.MSELoss()

# Define train loop

In [ ]:
def train(criterion, model, loader, optimizer):
    model.train()
    device = next(model.parameters()).device
    total_loss = 0
    for batch in loader:
        input_seq_batch = batch['input'].to(device)
        target_seq_batch = batch['target'].to(device)
        optimizer.zero_grad()
        predicted_output = model(input_seq_batch)
        loss = criterion(target_seq_batch, predicted_output)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    total_loss /= len(loader)
    return total_loss

## Train!

In [ ]:
for epoch in range(n_epochs):
    loss = train(criterion, model, loader, optimizer)
    print("Epoch {} -- Loss {:3E}".format(epoch, loss))